In [88]:
import numpy as np
from main import *

## Définition d'une version adaptative

In [18]:
def gradient_rho_adaptatif(fun, fun_der, U0, rho, tol,args):
    
    # Fonction permettant de minimiser la fonction f(U) par rapport au vecteur U 
    # Méthode : gradient à pas fixe
    # INPUTS :
    # - han_f   : handle vers la fonction à minimiser
    # - han_df  : handle vers le gradient de la fonction à minimiser
    # - U0      : vecteur initial 
    # - rho     : paramètre gérant l'amplitude des déplacement 
    # - tol     : tolérance pour définir le critère d'arrêt
    # OUTPUT : 
    # - GradResults : structure décrivant la solution


    itermax=10000  # nombre maximal d'itérations 
    xn=U0
    f=fun(xn,*args) # point initial de l'algorithme
    it=0         # compteur pour les itérations
    converged = False;
    
    while (~converged & (it < itermax)):
        it=it+1
        dfx=fun_der(xn,*args)# valeur courante de la fonction à minimiser
        
        xnp1=xn-rho*dfx
        fnp1 = fun(xnp1,*args)
        
        if fnp1 < f :
            
            if abs(fnp1-f)<tol:
                converged = True
                
            xn, f = xnp1, fnp1
            rho *= 2
        else :
            rho /= 2

    GradResults = {
            'initial_x':U0,
            'minimum':xnp1,
            'f_minimum':fnp1,
            'iterations':it,
            'converged':converged
            }
    return GradResults

In [23]:
gradient_rho_adaptatif(f1,df1,x0,rho=1,tol=1e-6,args=(B,S))

{'initial_x': array([1., 1., 1., 1., 1.]),
 'minimum': array([-0.67000244,  0.1425606 , -0.61584071,  0.47887395, -0.02041732]),
 'f_minimum': -1.8361443391078145,
 'iterations': 386,
 'converged': True}

## On va plot le nombre d'itération par rapport à rho

## Méthodes Quasi-Newton

In [24]:
from scipy.optimize import minimize

In [27]:
minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)

      fun: -1.836962311965238
 hess_inv: array([[ 0.5004328 , -0.21223272,  0.05691395, -0.28056012,  0.50919827],
       [-0.21223272,  0.22383377,  0.0370393 ,  0.11542843, -0.34043246],
       [ 0.05691395,  0.0370393 ,  0.1247386 , -0.04905887, -0.09108766],
       [-0.28056012,  0.11542843, -0.04905887,  0.20504998, -0.28961765],
       [ 0.50919827, -0.34043246, -0.09108766, -0.28961765,  0.77691404]])
      jac: array([-2.98023224e-08, -8.94069672e-08,  1.49011612e-08, -1.49011612e-08,
        1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 91
      nit: 10
     njev: 13
   status: 0
  success: True
        x: array([-0.69603139,  0.15793134, -0.61407083,  0.49414155, -0.05345803])

## Résolution analytique

In [29]:
np.linalg.eigvals(S)

array([45.48200751, 14.49627115,  0.34399006,  1.89569112,  7.21954015])

solution analytique existe

In [43]:
S_1 = np.linalg.inv(S)
solution = 0.5*S_1*B
np.abs(f1(solution, B, S) - minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)['fun'])

1.2434497875801753e-14

## Optimisation sous contraintes

In [44]:
minimize(fun = f1, x0 = x0, args=(B,S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: -0.13853161426327318
     jac: array([5.65140143e-01, 1.70478411e-03, 4.86906106e+00, 1.77649595e-03,
       2.70384220e-01])
 message: 'Optimization terminated successfully.'
    nfev: 65
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([4.70812343e-16, 1.26984957e-01, 8.12072123e-16, 1.94536141e-02,
       1.33997327e-16])

In [55]:
def f2(U,S):
    n=U.shape[0]
    U=np.matrix(U)
    U.shape=(n,1)
    fU = np.transpose(U) * S * U + np.transpose(U) * np.exp(U);
    return float(fU)

In [56]:
minimize(fun = f2, x0 = x0, args=(S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: 1.385691638709516e-15
     jac: array([1.00000018, 1.00000013, 1.00000035, 1.00000023, 1.00000022])
 message: 'Optimization terminated successfully.'
    nfev: 28
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.00000000e+00, 0.00000000e+00, 1.38569164e-15, 0.00000000e+00,
       0.00000000e+00])

## Optimisation sous contraintes et pénalisation

In [89]:
#fonction de pénalisation classique 
Beta = lambda u : np.sum(np.maximum(u-1, 0)**2 + (np.maximum(-u,0))**2)
print(Beta(x0), Beta(x0 - 2))

0.0 5.0


In [110]:
epsilon, start = 1/2, x0
for k in range(100):
    f1_penal = lambda U : f1(U,B,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f1_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f1(start, B , S))
print(start)

-0.138531737210628
[1.65128399e-17 1.26885911e-01 7.00775436e-11 1.94103243e-02
 5.20345553e-11]


In [109]:
epsilon, start = 1/2, x0
for k in range(1000):
    f2_penal = lambda U : f2(U,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f2_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f2(start , S))
print(start)

6.399087921480177e-11
[8.72575031e-17 8.72575031e-17 8.72575031e-17 8.72575031e-17
 6.39905301e-11]


## Méthodes duales 